# 使用MindSpore训练一个简单网络

本节通过MindSpore的API快速实现一个简单的神经网络模型，使用MNIST数据集。

## 1、实验目的

* 学会手写数字图像的数据载入及处理。
* 掌握搭建全连接神经网络。
* 掌握分类任务的训练流程。

## 2、全连接神经网络原理介绍

全连接神经网络，也可称为稠密连接网络，是一种多层的感知机结构，每一层的每一个节点都与上下层节点全部连接，这就是”全连接“的由来。整个全连接神经网络分为输入层，隐藏层和输出层，其中隐藏层可以更好的分离数据的特征，但是过多的隐藏层会导致过拟合问题。

在模型训练过程中包含前向传播，反向传播，和模型更新步骤。

其中，当前馈神经网络接收输入$x$并产生输出$\hat y$时，信息通过网络向前流动。输入$x$提供初始信息，经过每一层的隐藏单元最终生成输出$\hat y$，称为前向传播。此计算过程等同于线性回归计算，即给每一个输入向量x分配权值，计算出一个结果向量$\hat y$。同时，为了使神经网络具有非线性特点，引入激活函数来处理线性变换得到的数值。对于单个神经元来讲：
- 线性变换（加权和偏置）：$\hat y=w^Tx+b$
- 非线性变换（激活函数）：例如$\delta(x)=\frac{1}{1+e^{-x}}$

前向传播可以持续向前指导产生标量成本函数，成本函数的信息通过网络向后流动，计算梯度优化网络，称为反向传播。反向传播在模型训练时通常被划分为梯度计算和参数优化两个部分。由于网络模型中可以定义多个隐藏层，需要递归的使用链式法则来实现反向传播。<br>

全连接神经网络图示：
![](./image/img2.png)

## 3、实验环境


在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理


### 4.1 数据准备

MNIST数据集是机器学习领域中经典的数据集，由60,000个训练样本和10,000个测试样本组成，每个样本是28\*28像素的灰度手写数字图片，共10类（0-9）。整个数据集约为50M。

下载的数据集文件的目录结构如下：

```text
./MNIST_Data
├── test
│   ├── t10k-images-idx3-ubyte
│   └── t10k-labels-idx1-ubyte
└── train
    ├── train-images-idx3-ubyte
    └── train-labels-idx1-ubyte
```
MNIST示例

![](./image/img1.png)

MindSpore提供基于Pipeline的数据引擎，通过数据集（Dataset）和数据变换（Transforms）实现高效的数据预处理。在本教程中，我们使用Mnist数据集，自动下载完成后，使用mindspore.dataset提供的数据变换进行预处理。



>此处的示例代码依赖download，可使用命令pip install download安装。如本文档以Notebook运行时，完成安装后需要重启kernel才能执行后续代码。

In [6]:
# 从开放数据集中下载MNIST数据集
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:05<00:00, 1.84MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


### 4.2 数据加载

导入库

In [ ]:
# MindSpore库
import mindspore
# 神经网络模块
from mindspore import nn
# 常见算子操作
from mindspore import ops
# 图像增强模块
from mindspore.dataset import vision
# 通用数据增强
from mindspore.dataset import transforms
# 读取和解析Manifest数据文件构建数据集
from mindspore.dataset import MnistDataset

设置超参数

In [1]:
BATCH_SIZE= 64       # batch的大小
LEARNING_RATE = 1e-2 # 学习率
EPOCH = 3            # 迭代次数

数据下载完成后，获得数据集对象。

In [7]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

MindSpore的dataset使用数据处理流水线（Data Processing Pipeline），需指定map、batch、shuffle等操作。这里我们使用map对图像数据及标签进行变换处理，然后将处理好的数据集打包为大小为64的batch。

In [9]:
def datapipe(dataset, batch_size):
    image_transforms = [
        # 基于给定的缩放和平移因子调整图像的像素大小。输出图像的像素大小为：output = image * rescale + shift。
        # 此处rescale取1.0 / 255.0，shift取0
        vision.Rescale(1.0 / 255.0, 0),
        # 正则化 均值为0.1307，标准差为0.3081（查自官网）
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        # 将输入图像的shape从 <H, W, C> 转换为 <C, H, W>
        vision.HWC2CHW()
    ]
    # 将输入的Tensor转换为指定的数据类型。
    label_transform = transforms.TypeCast(mindspore.int32)

    # map给定一组数据增强列表，按顺序将数据增强作用在数据集对象上。
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    # 将数据集中连续 batch_size 条数据组合为一个批数据
    dataset = dataset.batch(batch_size)
    return dataset
# 对数据集进行transfrom和batch
train_dataset = datapipe(train_dataset, BATCH_SIZE)
test_dataset = datapipe(test_dataset, BATCH_SIZE)

## 5、模型构建


mindspore.nn类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承nn.Cell类，并重写__init__方法和construct方法。__init__包含所有网络层的定义，construct中包含数据（Tensor）的变换过程（即计算图的构造过程）。

In [11]:
# 定义模型
# MindSpore 中提供用户通过继承 nn.Cell 来方便用户创建和执行自己的网络
class Network(nn.Cell): 
    # 自定义的网络中，需要在__init__构造函数中申明各个层的定义
    def __init__(self): 
         # 继承父类nn.cell的__init__方法
        super().__init__()         
        # nn.Flatten为输入展成平图层，即去掉那些空的维度
        self.flatten = nn.Flatten()
        # 使用SequentialCell对网络进行管理
        self.dense_relu_sequential = nn.SequentialCell(
            # nn.Dense为致密连接层，它的第一个参数为输入层的维度，第二个参数为输出的维度，
            # 第三个参数为神经网络可训练参数W权重矩阵的初始化方式，默认为normal
            # nn.ReLU()非线性激活函数，它往往比论文中的sigmoid激活函数具有更好的效益
            nn.Dense(28 * 28, 512), # 致密连接层 输入28*28 输出512
            nn.ReLU(),              # ReLU层
            nn.Dense(512, 512),     # 致密连接层 输入512 输出512
            nn.ReLU(),              # ReLu层
            nn.Dense(512, 10)       # 致密连接层 输入512 输出10
        )
    # 在construct中实现层之间的连接关系，完成神经网络的前向构造
    def construct(self, x):
         #调用init中定义的self.flatten()方法 
        x = self.flatten(x)
        #调用init中的self.dense_relu_sequential()方法
        logits = self.dense_relu_sequential(x)
        # 返回模型
        return logits
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


## 6、模型训练

In [12]:
# 实例化损失函数和优化器
# 计算预测值和目标值之间的交叉熵损失
loss_fn = nn.CrossEntropyLoss() 
#构建一个Optimizer对象，能够保持当前参数状态并基于计算得到的梯度进行参数更新 此处使用随机梯度下降算法
optimizer = nn.SGD(model.trainable_params(), learning_rate=LEARNING_RATE) 

在模型训练中，一个完整的训练过程（step）需要实现以下三步：
1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。
2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。
3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：
1. 正向计算函数定义。
2. 通过函数变换获得梯度计算函数。
3. 训练函数定义，执行正向计算、反向传播和参数优化。

In [13]:
def train(model, dataset, loss_fn, optimizer):
    # 定义 forward 函数
    def forward_fn(data, label):
        # 将数据载入模型
        logits = model(data)
        # 根据模型训练获取损失函数值
        loss = loss_fn(logits, label)
        return loss, logits
    # 调用梯度函数，value_and_grad()为生成求导函数，用于计算给定函数的正向计算结果和梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    # 定义一步训练的函数
    def train_step(data, label):
        # 计算梯度，记录变量是怎么来的
        (loss, _), grads = grad_fn(data, label)
        # 获得损失 depend用来处理操作间的依赖关系
        loss = ops.depend(loss, optimizer(grads))
        return loss
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        # 批量训练获得损失值
        loss = train_step(data, label)
        # 当完成所有数据样本的训练
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

除训练外，我们定义测试函数，用来评估模型的性能。

In [14]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator(): # 遍历所有测试样本数据
        pred = model(data)                              # 根据已训练模型获取预测值
        total += len(data)                              # 统计样本数
        test_loss += loss_fn(pred, label).asnumpy()     # 统计样本损失值
        correct += (pred.argmax(1) == label).asnumpy().sum()# 统计预测正确的样本个数
    test_loss /= num_batches                              # 求得平均损失
    correct /= total                                      # 计算accuracy
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需多次迭代数据集，一次完整的迭代称为一轮（epoch）。在每一轮，遍历训练集进行训练，结束后使用测试集进行预测。打印每一轮的loss值和预测准确率（Accuracy），可以看到loss在不断下降，Accuracy在不断提高。

In [15]:
for t in range(EPOCH):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)      # 训练模型
    test(model, test_dataset, loss_fn)                   # 测试模型
print("Done!")

Epoch 1
-------------------------------
loss: 2.302374  [  0/938]
loss: 2.289070  [100/938]
loss: 2.258706  [200/938]
loss: 2.162465  [300/938]
loss: 1.990072  [400/938]
loss: 1.491828  [500/938]
loss: 1.056657  [600/938]
loss: 0.791395  [700/938]
loss: 0.747034  [800/938]
loss: 0.758159  [900/938]
Test: 
 Accuracy: 85.5%, Avg loss: 0.518855 

Epoch 2
-------------------------------
loss: 0.534654  [  0/938]
loss: 0.645424  [100/938]
loss: 0.438396  [200/938]
loss: 0.397434  [300/938]
loss: 0.506358  [400/938]
loss: 0.280990  [500/938]
loss: 0.275507  [600/938]
loss: 0.341971  [700/938]
loss: 0.279664  [800/938]
loss: 0.425316  [900/938]
Test: 
 Accuracy: 90.0%, Avg loss: 0.336232 

Epoch 3
-------------------------------
loss: 0.431660  [  0/938]
loss: 0.299660  [100/938]
loss: 0.493997  [200/938]
loss: 0.144846  [300/938]
loss: 0.149156  [400/938]
loss: 0.377380  [500/938]
loss: 0.335227  [600/938]
loss: 0.215083  [700/938]
loss: 0.125211  [800/938]
loss: 0.312093  [900/938]
Test: 
 

## 7、模型预测

###  保存模型
模型训练完成后，需要将其参数进行保存

In [16]:
# 保存checkpoint时的配置策略
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


### 加载模型
加载保存的权重分为两步：
1. 重新实例化模型对象，构造模型。
2. 加载模型参数，并将其加载至模型上。

In [17]:
# 实例化一个随机初始化的模型 
model = Network()
# 加载检查点，加载参数到模型 
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)

[]


>param_not_load是未被加载的参数列表，为空时代表所有参数均加载成功。

加载后的模型可以直接用于预测推理。

In [18]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[8 3 2 1 0 1 6 7 2 2]", Actual: "[8 3 2 1 0 1 6 7 2 2]"
